In [1]:
import pandas as pd

# CSV dosyasının yolunu tanımla
file_path = 'MajorCities.csv'  # 'path_to_your_file' kısmını dosyanızın konumuyla değiştirin

# Dosyayı oku
cities_df = pd.read_csv(file_path)

# 'dt' sütunundan yılı çıkar ve yeni bir 'Year' sütunu oluştur
cities_df['Year'] = pd.to_datetime(cities_df['dt']).dt.year

# 1920 ve 1970 yılları arasındaki verileri filtrele
filtered_cities_df = cities_df[(cities_df['Year'] >= 1920) & (cities_df['Year'] <= 1970)]

# Her bir şehir için yıllık ortalama sıcaklığı hesapla
yearly_avg_temps = (
    filtered_cities_df.groupby(['City', 'Country', 'Latitude', 'Longitude', 'Year'])['AverageTemperature']
    .mean()
    .reset_index()
)

# Sonucun ilk beş satırını görüntüle
print(yearly_avg_temps.head())


      City        Country  Latitude  Longitude  Year  AverageTemperature
0  Abidjan  Côte D'Ivoire      5.63      -3.23  1920           26.324167
1  Abidjan  Côte D'Ivoire      5.63      -3.23  1921           26.420250
2  Abidjan  Côte D'Ivoire      5.63      -3.23  1922           26.229167
3  Abidjan  Côte D'Ivoire      5.63      -3.23  1923           25.962167
4  Abidjan  Côte D'Ivoire      5.63      -3.23  1924           26.092167


In [2]:
# "Peking" ve "Los Angeles" şehirlerinin koordinatlarını bulalım
peking_coords = yearly_avg_temps[yearly_avg_temps['City'] == 'Peking'][['Latitude', 'Longitude']].iloc[0]
los_angeles_coords = yearly_avg_temps[yearly_avg_temps['City'] == 'Los Angeles'][['Latitude', 'Longitude']].iloc[0]

peking_coords, los_angeles_coords

(Latitude      39.38
 Longitude    116.53
 Name: 3723, dtype: float64,
 Latitude      34.56
 Longitude   -118.70
 Name: 2805, dtype: float64)

In [3]:
from math import radians, cos, sin, asin, sqrt

# Haversine formülü kullanarak iki koordinat arasındaki mesafeyi hesaplayan fonksiyon
def haversine(lon1, lat1, lon2, lat2):
    # Radyan cinsinden koordinatları dönüştür
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Boylam ve enlem farkları
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    # Haversine formülü
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371  # Dünya'nın yarıçapı (km cinsinden)
    return c * r

# Mevcut konumun koordinatlarından en yakın şehirlerin listesini döndüren fonksiyon
def find_nearest_cities(current_coords, cities_data, n=3):
    # Haversine formülü ile mesafeleri hesapla
    distances = cities_data.apply(
        lambda row: haversine(current_coords['Longitude'], current_coords['Latitude'], row['Longitude'], row['Latitude']),
        axis=1
    )
    
    # En yakın şehirleri döndürürken, mevcut konumu (0 mesafede) hariç tut
    nearest_cities = cities_data.loc[distances.nlargest(len(distances) - 1).nsmallest(n).index]
    return nearest_cities

# Her şehir için yıllık ortalama sıcaklıkların genel ortalamasını hesaplayalım
city_avg_temps = yearly_avg_temps.groupby(['City', 'Country', 'Latitude', 'Longitude'])['AverageTemperature'].mean().reset_index()

# Artık her şehir için sadece bir kayıt içeren yeni veri setini kullanarak Pekin'e en yakın şehirleri bulalım
nearest_to_peking = find_nearest_cities(peking_coords, city_avg_temps)
print(nearest_to_peking)


        City Country  Latitude  Longitude  AverageTemperature
73    Peking   China     39.38     116.53           11.915804
93  Tangshan   China     37.78     113.90           11.728559
24    Dalian   China     39.38     120.69           10.246918


In [5]:
# suggest_route fonksiyonunun güncellenmiş hali
def suggest_route(start_record, end_record, cities_data):
    # Start with the initial city
    route = [start_record['City']]
    current_record = start_record
    visited_cities = set(route)

    # Loop until the current city is the destination city
    while current_record['City'] != end_record['City']:
        # Exclude visited cities and the current city from the search
        unvisited_cities = cities_data[~cities_data['City'].isin(visited_cities)]

        # Find the nearest cities that have not been visited
        nearest_cities = find_nearest_cities(current_record, unvisited_cities, n=3)

        # If there are no unvisited cities left, break the loop (to avoid an infinite loop)
        if nearest_cities.empty:
            print("No more unvisited cities to go to. Ending route search.")
            break

        # Select the nearest city with the highest average temperature
        next_city_record = nearest_cities.loc[nearest_cities['AverageTemperature'].idxmax()]

        # If the destination city is the next city, add it to the route and break the loop
        if next_city_record['City'] == end_record['City']:
            route.append(next_city_record['City'])
            break

        # Add the selected city to the route and mark it as visited
        route.append(next_city_record['City'])
        visited_cities.add(next_city_record['City'])
        
        # Update the current city record
        current_record = next_city_record

    return route

# Rota önerisini al
route = suggest_route(peking_record, los_angeles_record, city_avg_temps)

# Rota çıktısını yazdır
print(route)


['Peking', 'Jinan', 'Tianjin', 'Dalian', 'Tangshan', 'Wuhan', 'Guangzhou', 'Taipei', 'Manila', 'Chongqing', 'Nanjing', 'Xian', 'Shanghai', 'Taiyuan', 'Seoul', 'Nagoya', 'Shenyang', 'Tokyo', 'Ho Chi Minh City', 'Rangoon', 'Madras', 'Hyderabad', 'Bombay', 'Ahmadabad', 'Karachi', 'Jaipur', 'Delhi', 'Kanpur', 'Calcutta', 'Nagpur', 'New Delhi', 'Surat', 'Lakhnau', 'Dhaka', 'Bangalore', 'Bangkok', 'Surabaya', 'Singapore', 'Pune', 'Faisalabad', 'Baghdad', 'Riyadh', 'Cairo', 'Alexandria', 'Aleppo', 'Gizeh', 'Jiddah', 'Izmir', 'Rome', 'Istanbul', 'Berlin', 'London', 'Madrid', 'Paris', 'Kiev', 'Mashhad', 'Lahore', 'Chengdu', 'Jakarta', 'Sydney', 'Durban', 'Dar Es Salaam', 'Mogadishu', 'Umm Durman', 'Kano', 'Lagos', 'Abidjan', 'Dakar', 'Fortaleza', 'Rio De Janeiro', 'Salvador', 'Brasília', 'São Paulo', 'Bogotá', 'Santo Domingo', 'New York', 'Mexico', 'Cali', 'Belo Horizonte', 'Ibadan', 'Luanda', 'Harare', 'Kinshasa', 'Addis Abeba', 'Cape Town', 'Casablanca', 'Montreal', 'Los Angeles']


In [6]:
# Basitleştirilmiş bir rota önerisi algoritması
def simple_suggest_route(start_record, end_record, cities_data):
    route = [start_record['City']]
    current_record = start_record
    visited_cities = set(route)
    
    while current_record['City'] != end_record['City']:
        # Exclude visited cities and the current city from the search
        unvisited_cities = cities_data[~cities_data['City'].isin(visited_cities)]
        
        # Instead of finding the nearest cities based on actual distance,
        # we'll select cities with the closest coordinates
        closest_cities = unvisited_cities.iloc[
            (unvisited_cities['Latitude'] - current_record['Latitude']).abs().argsort()[:3]
        ]

        # If there are no unvisited cities left, break the loop (to avoid an infinite loop)
        if closest_cities.empty:
            print("No more unvisited cities to go to. Ending route search.")
            break

        # Select the city with the highest average temperature from the closest cities
        next_city_record = closest_cities.loc[closest_cities['AverageTemperature'].idxmax()]

        # If the destination city is the next city, add it to the route and break the loop
        if next_city_record['City'] == end_record['City']:
            route.append(next_city_record['City'])
            break

        # Add the selected city to the route and mark it as visited
        route.append(next_city_record['City'])
        visited_cities.add(next_city_record['City'])
        
        # Update the current city record
        current_record = next_city_record

    return route

# Let's try the simple_suggest_route function and compare the result with the previous one.
simple_route = simple_suggest_route(peking_record, los_angeles_record, city_avg_temps)
simple_route


['Peking',
 'Tianjin',
 'Izmir',
 'Tangshan',
 'Seoul',
 'Ankara',
 'Dalian',
 'Istanbul',
 'Madrid',
 'Rome',
 'Chicago',
 'New York',
 'Taiyuan',
 'Aleppo',
 'Jinan',
 'Tokyo',
 'Nagoya',
 'Los Angeles']